In [ ]:
push!(LOAD_PATH, "..")
using atoms
using lj
reload("pbc")

In [ ]:
at = bulk("Cu"; crystalstructure="fcc", cubic=true)
at = repeat(at, (2, 2, 2))

In [ ]:
using lj
using pbc

rho = length(at)/det(get_cell(at))
pbc.set_density(length(at), rho)
println(lj.info())

e = lj.energy(get_positions(at))

In [ ]:
using PyCall
@pyimport quippy.potential as potential
lj_pot = potential.Potential("IP LJ"; param_str="""
<LJ_params n_types="3" label="default">
<per_type_data type="1" atomic_num="29" />
<per_pair_data type1="1" type2="1" sigma="1.0" eps6="1.0" eps12="1.0" cutoff="3.0" energy_shift="F" linear_force_shift="F" />
</LJ_params>""")
set_calculator(at, lj_pot)
get_potential_energy(at)

In [ ]:
# compute neighbour list using matscipy
i, j, rij = neighbours(at, "ijd", sw[:cutoff]())

# estimate nearest neighbour distance
r_nn = median([ minimum(rij[i .== n]) for n = 1:length(at) ])

In [ ]:
p = [0. 0. 0.;
     0. 0. 0.]
p = p' # NB: Gabor's LJ module expects (3, N) arrays not (N, 3) like ASE
X = linspace(1, 3)
Ej = zeros(length(X))
for (i, x) in enumerate(X)
    p[1,2] = x
    Ej[i] = lj.energy(p)
end

In [ ]:
p = [0. 0. 0.;
     0. 0. 0.]
dimerq = convert(Atoms, atoms.ase.Atoms("Cu2"; positions=p, cell=100.0*eye(3), pbc=false))
set_calculator(dimerq, lj_pot)
Eq = zeros(length(X))
for (i, x) in enumerate(X)
    p[2,1] = x
    set_positions!(dimerq, p)
    Eq[i] = get_potential_energy(dimerq)
end

In [ ]:
@pyimport ase.calculators.lj as ase_lj
lj_calc = pycall(ase_lj.LennardJones, PyAny)
lj_calc[:set](; epsilon=1.0/4.0, sigma=1.0, rc=3.0)
p = [0. 0. 0.;
     0. 0. 0.]
dimera = convert(Atoms, atoms.ase.Atoms("Cu2"; positions=p, cell=100.0*eye(3), pbc=false))
set_calculator(dimera, lj_calc)
Ea = zeros(length(X))
for (i, x) in enumerate(X)
    p[2,1] = x
    set_positions!(dimera, p)
    Ea[i] = get_potential_energy(dimera)
end

In [ ]:
using PyPlot

In [ ]:
clf()
plot(X, Ej; label="Julia")
plot(X, Ea; label="ASE")
plot(X, Eq; label="QUIP")
legend()

In [ ]:
@time lj.energy(p')

In [ ]:
@time get_potential_energy(dimerq)

In [ ]:
@time get_potential_energy(dimera)